Saai Narayann Maddu

210968036

Section - A1

In [3]:
import sys
import gym
import numpy as np

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
env = gym.make('FrozenLake-v1',is_slippery=True)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [6]:
def calculate_state_value(environment, current_state, value_matrix, discount_factor):
    """
    Calculates the state values for a given state using the Bellman equation.

    Args:
    - environment: The environment object representing the Markov Decision Process (MDP).
    - current_state: The current state for which the state value is to be calculated.
    - value_matrix: A matrix containing the current estimated values for each state.
    - discount_factor: Discount factor for future rewards.

    Returns:
    - action_values: An array containing the calculated value for each action in the current state.
    """

    # Get the number of possible actions in the environment
    num_actions = environment.action_space.n

    # Initialize an array to store the value of each action
    action_values = np.zeros(shape=num_actions)

    # Iterate over each action
    for action in range(num_actions):
        # Iterate over each possible transition from the current state for the given action
        for transition_prob, next_state, reward, done in environment.env.P[current_state][action]:
            # Calculate the expected value for the action using the Bellman equation
            action_values[action] += transition_prob * (reward + discount_factor * value_matrix[next_state])

    return action_values


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def policy_evaluation(policy, environment, discount_factor=1.0, convergence_threshold=1e-9, max_iterations=1000):
    """
    Performs policy evaluation to estimate the state values for a given policy.

    Args:
    - policy: The policy matrix specifying the probability of taking each action in each state.
    - environment: The environment object representing the Markov Decision Process (MDP).
    - discount_factor: Discount factor for future rewards.
    - convergence_threshold: Threshold for determining convergence in state value estimation.
    - max_iterations: Maximum number of iterations for policy evaluation.

    Returns:
    - state_values: An array containing the estimated value for each state under the given policy.
    """

    # Get the number of states in the environment
    num_states = environment.observation_space.n

    # Initialize state values to zeros
    state_values = np.zeros(shape=num_states)

    # Counter for iterations
    evaluation_iterations = 1

    # Iterate until convergence or maximum iterations reached
    for iteration in range(int(max_iterations)):
        delta = 0  # Initialize the maximum change in value for this iteration
        # Iterate over all states
        for current_state in range(num_states):
            new_state_value = 0  # Initialize the new value for the current state
            # Iterate over all possible actions in the current state
            for action, action_probability in enumerate(policy[current_state]):
                # Iterate over all possible next states and their probabilities
                for state_probability, next_state, reward, done in environment.P[current_state][action]:
                    # Update the new state value based on the action, next state, and reward
                    new_state_value += action_probability * state_probability * (reward + discount_factor * state_values[next_state])
            # Update the maximum change in value
            delta = max(delta, np.abs(state_values[current_state] - new_state_value))
            # Update the state value with the new value
            state_values[current_state] = new_state_value

        evaluation_iterations += 1

        # Check for convergence
        if delta < convergence_threshold:
            print(f'Policy evaluation terminated after {evaluation_iterations} iterations.\n')
            return state_values

    # If max_iterations reached without convergence, return the current state values
    print(f'Policy evaluation reached maximum iterations ({max_iterations}).')
    return state_values


In [8]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1000):
    """
    Implements the policy iteration algorithm to find the optimal policy for the given environment.

    Args:
    - environment: The environment object representing the Markov Decision Process (MDP).
    - discount_factor: Discount factor for future rewards.
    - max_iterations: Maximum number of iterations for policy iteration.

    Returns:
    - optimal_policy: The optimal policy matrix.
    - optimal_value_function: The value function corresponding to the optimal policy.
    """

    num_states = environment.observation_space.n
    num_actions = environment.action_space.n

    # Initialize a uniform random policy matrix
    policy_matrix = np.ones(shape=[num_states, num_actions]) / num_actions

    # Counter for evaluated policies
    evaluated_policies_count = 1

    # Iterate until convergence or maximum iterations reached
    for iteration in range(int(max_iterations)):
        stable_policy = False

        # Evaluate the current policy
        value_function = policy_evaluation(policy_matrix, environment, discount_factor)

        # Improve the policy
        for current_state in range(num_states):
            current_action = np.argmax(policy_matrix[current_state])
            action_values = calculate_state_value(environment, current_state, value_function, discount_factor)
            best_action = np.argmax(action_values)

            # Check if the policy is stable
            if current_action != best_action:
                stable_policy = True

            # Update the policy matrix to select the best action in each state
            policy_matrix[current_state] = np.eye(num_actions)[best_action]

        evaluated_policies_count += 1

        # If the policy is stable, return the optimal policy and value function
        if not stable_policy:
            print(f'Found a stable policy after {evaluated_policies_count:,} evaluations.\n')
            return policy_matrix, value_function

    # If maximum iterations reached without convergence, return the current policy and value function
    print(f'Maximum iterations reached ({max_iterations}). Returning current policy.\n')
    return policy_matrix, value_function


In [9]:
def value_iteration(environment, discount_factor=1e-1, convergence_threshold=1e-9, max_iterations=1e4):
    """
    Implements the value iteration algorithm to find the optimal policy for the given environment.

    Args:
    - environment: The environment object representing the Markov Decision Process (MDP).
    - discount_factor: Discount factor for future rewards.
    - convergence_threshold: Threshold for determining convergence in value iteration.
    - max_iterations: Maximum number of iterations for value iteration.

    Returns:
    - optimal_policy: The optimal policy matrix.
    - optimal_state_values: The value function corresponding to the optimal policy.
    """

    # Initialize state values to zeros
    state_values = np.zeros(environment.observation_space.n)

    # Iterate until convergence or maximum iterations reached
    for iteration in range(int(max_iterations)):
        delta = 0  # Initialize the maximum change in value for this iteration

        # Update state values
        for current_state in range(environment.observation_space.n):
            action_values = calculate_state_value(environment, current_state, state_values, discount_factor)
            best_action_value = np.max(action_values)
            delta = max(delta, np.abs(state_values[current_state] - best_action_value))
            state_values[current_state] = best_action_value

        # Check for convergence
        if delta < convergence_threshold:
            print(f'\nValue iteration converged at iteration #{iteration+1:,}')
            break

    # Create the optimal policy matrix based on the computed state values
    optimal_policy = np.zeros(shape=[environment.observation_space.n, environment.action_space.n])

    # Determine the best action for each state based on the computed state values
    for current_state in range(environment.observation_space.n):
        action_values = calculate_state_value(environment, current_state, state_values, discount_factor)
        best_action = np.argmax(action_values)
        optimal_policy[current_state, best_action] = 1.0

    return optimal_policy, state_values


In [10]:
def evaluate_policy_performance(env, num_episodes, policy, max_actions=100, render=False):
    """
    Plays episodes using the given policy and evaluates its performance.

    Args:
    - env: The environment object representing the Markov Decision Process (MDP).
    - num_episodes: Number of episodes to play.
    - policy: The policy matrix specifying the probability of taking each action in each state.
    - max_actions: Maximum number of actions allowed per episode.
    - render: Whether to render the environment.

    Returns:
    - total_wins: Total number of episodes won.
    - total_rewards: Total rewards obtained across all episodes.
    - average_reward: Average reward per episode.
    - average_actions: Average number of actions taken per episode.
    """

    # Initialize counters
    total_wins = 0
    total_rewards, total_actions = 0, 0

    # Play episodes
    for episode in range(num_episodes):
        current_state = env.reset()
        episode_done, actions_taken = False, 0

        # Play until the episode is done or maximum actions are taken
        while actions_taken < max_actions:
            # Choose action based on the policy
            selected_action = np.argmax(policy[current_state])
            next_state, reward, episode_done, _ = env.step(selected_action)

            # Render the environment if required
            if render:
                env.render()

            # Update counters
            actions_taken += 1
            total_rewards += reward
            current_state = next_state

            # Break if episode is done
            if episode_done:
                total_wins += 1
                break

        total_actions += actions_taken

    # Print total rewards and max actions taken in the last episode
    print(f'Total rewards: {total_rewards:,}\tMax actions: {actions_taken:,}')

    # Calculate average reward and average actions per episode
    average_reward = total_rewards / num_episodes
    average_actions = total_actions / num_episodes

    print('')

    return total_wins, total_rewards, average_reward, average_actions


In [11]:
num_episodes = 1000

def evaluate_policies_and_agents(env):
    """
    Evaluates different policies and agents using policy iteration and value iteration algorithms.

    Args:
    - env: The environment object representing the Markov Decision Process (MDP).

    Returns:
    - total_rewards_list: A list containing the total rewards obtained for each evaluated policy/agent.
    """

    total_rewards_list = []

    action_mapping = {
        0: '\u2191',  # up
        1: '\u2192',  # right
        2: '\u2193',  # down
        3: '\u2190'   # left
    }

    iteration_methods = [
        ('Policy Iteration', policy_iteration),
        ('Value Iteration', value_iteration)
    ]

    for method_name, method_func in iteration_methods:
        policy_matrix, value_function = method_func(env)

        print(f'Final policy using {method_name}:')
        print(' '.join([action_mapping[action] for action in np.argmax(policy_matrix, axis=1)]))

        total_wins, total_rewards, avg_reward, avg_actions = evaluate_policy_performance(env, num_episodes, policy_matrix)
        total_rewards_list.append(total_rewards)

        print(f'Number of wins = {total_wins:,}')
        print(f'Average reward = {avg_reward:.2f}')
        print(f'Average actions = {avg_actions:.2f}')

    return total_rewards_list


In [13]:
rewards = evaluate_policies_and_agents(env)

Policy evaluation terminated after 66 iterations.

Policy evaluation terminated after 417 iterations.

Policy evaluation terminated after 527 iterations.

Found a stable policy after 4 evaluations.

Final policy using Policy Iteration:
↑ ← ← ← ↑ ↑ ↑ ↑ ← → ↑ ↑ ↑ ↓ → ↑


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Total rewards: 722.0	Max actions: 47

Number of wins = 1,000
Average reward = 0.72
Average actions = 45.84

Value iteration converged at iteration #8
Final policy using Value Iteration:
→ ← ↓ ← ↑ ↑ ↑ ↑ ← → ↑ ↑ ↑ ↓ → ↑
Total rewards: 446.0	Max actions: 19

Number of wins = 1,000
Average reward = 0.45
Average actions = 26.97
